<a href="https://colab.research.google.com/github/rafaela-arias/colabandamal/blob/main/RAFA_TA047R_TP2_GRUPO03_CHPX_ENTREGA_N2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Grupo 3 - Trabajo Práctico 2

**Integrantes:**
- ARIAS, Rafaela Pilar. 112272. rparias@fi.uba.ar.
- BARCALA ROCA, Ramiro. 112478. rbarcala@fi.uba.ar.
- LEMA, Mora. 111831. molema@fi.uba.ar.
- IENCO, Lara Eliana. 111921. lienco@fi.uba.ar.


###Inicialización del entorno


In [1]:
# Importar las bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento de texto
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [2]:
#librerias necesarias para emmbedding
!pip install torch
!pip install torchtext
!pip install numpy


In [3]:
!pip install nltk

In [4]:
import nltk

In [5]:
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#Cargamos los datos al Notebook

!gdown 1rbGVg-g0_H4UxviHdM3KFEHv5_DjJd-P
!gdown 19RRGPOcV-gQojb7G3wCtu1pY2Q4fQs_y

Downloading...
From: https://drive.google.com/uc?id=1rbGVg-g0_H4UxviHdM3KFEHv5_DjJd-P
To: /content/test.csv
100% 11.1M/11.1M [00:00<00:00, 60.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=19RRGPOcV-gQojb7G3wCtu1pY2Q4fQs_y
To: /content/train.csv
100% 72.2M/72.2M [00:01<00:00, 46.0MB/s]


In [7]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

print("Train - Columnas:", train_df.shape[1], "Filas:", train_df.shape[0])
print("Test - Columnas:", test_df.shape[1], "Filas:", test_df.shape[0])

print("\nPrimeras filas del train:")
display(train_df.head())

print("\nColumnas del train:")
print(train_df.columns)

print("\nDistribución de clases:")
print(train_df['sentimiento'].value_counts())


Train - Columnas: 3 Filas: 50000
Test - Columnas: 2 Filas: 8599

Primeras filas del train:


,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo



Columnas del train:
Index(['ID', 'review_es', 'sentimiento'], dtype='object')

Distribución de clases:
sentimiento
positivo    25000
negativo    25000
Name: count, dtype: int64


Observamos la cantidad de filas nulas en ambos conjuntos (train y test) y la cantidad de filas duplicadas

In [8]:
print("Cantidad total de valores NaN en el dataset train:", train_df.isnull().sum().sum())
print("Cantidad total de valores NaN en el dataset test:", test_df.isnull().sum().sum())

Cantidad total de valores NaN en el dataset train: 0
Cantidad total de valores NaN en el dataset test: 0


In [9]:
duplicadas_train = train_df.duplicated().sum()
print("Cantidad de filas duplicadas en el dataset train:", duplicadas_train)

duplicadas_test = test_df.duplicated().sum()
print("Cantidad de filas duplicadas en el dataset test:", duplicadas_test)

Cantidad de filas duplicadas en el dataset train: 0
Cantidad de filas duplicadas en el dataset test: 0


Al no haber duplicadas ni filas con NaN, no tuvimos que hacer nada al respecto.

###Preprocesamiento de los datos


###Limpieza y Tokenización


Veamos si las clases están balanceadas

In [10]:
train_df['sentimiento'].value_counts()

,count
sentimiento,
positivo,25000
negativo,25000


Separamos el dataset en entrenamiento 80% y test 20%

In [11]:
X = train_df['review_es']
y = train_df['sentimiento']

#Stratify para balancear la división
X_train_text, X_val_text, y_train, y_val = train_test_split(X,
                                                            y,
                                                            test_size = 0.2,
                                                            random_state = 42,
                                                            stratify = y
)

In [12]:
y_train.value_counts()

,count
sentimiento,
positivo,20000
negativo,20000


In [13]:
y_val.value_counts()

,count
sentimiento,
negativo,5000
positivo,5000


In [13]:
def clean_text(text):
    text = text.lower()
    #re.sub(patron, remplazo, string) --> Devuelve el string con el patron reemplazado

    text = re.sub(r'([.,!?;:])', r' \1 ', text)  #Añade espacio después de puntos y comas si no hay (para que no se junten palabras)
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text) #Reemplaza todo lo que no sea letra o espacio
    text = re.sub(r'\s+[a-záéíóúüñ]\s+', ' ', text)  #Elimina letras sueltas entre espacios
    text = re.sub(r'\s+', ' ', text) #Y si se me juntaron varios espacios, los mergeo en uno
    return text.strip()


Descargamos el modelo de spaCy para español para usarlo en la función de Lemmatización

In [14]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 61.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [15]:
import spacy

#Cargar el modelo de español
nlp = spacy.load("es_core_news_sm")

#Función de lematización
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.lemma_.isalpha()])

Integramos clean_text y lemmatize_text en una función

In [16]:
def clean_and_lemmatize(text):
    cleaned = clean_text(text)
    lemmatized = lemmatize_text(cleaned)
    return lemmatized

In [17]:
#Defino la stoplist
stoplist = stopwords.words("spanish")
stoplist[:15]

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con']

In [18]:
def tokenizer(text):
    tokens = word_tokenize(text, language="spanish")
    return [
        t.lower()
        for t in tokens
        if t.isalpha()
        and t.lower() not in stoplist
        and 2 <= len(t) <= 20
        and len(set(t.lower())) > 1  #Descarta tokens con todas las letras iguales, ej: "aa", "aaa"
    ]

In [19]:

#Para chequear que estamos haciendo las cosas bien
ejemplo = X_train_text.sample(1).iloc[0]

print("Crítica original:")
print(ejemplo)
print("\nTexto limpio:")
print(clean_text(ejemplo))
print("\nTokens:")
print(tokenizer(clean_text(ejemplo)))

Crítica original:
*** SPOILERS *** Esta película: llamada Comida viva aquí en el Reino Unido es posiblemente la peor película que haya hecho y es brillante solo para eso.Una chica rica sexy se adapta a un hombre de acción robusto para buscar a su hermana en la Nueva Guinea Jungle.Lo que sigue es una mezcla indescriptible de canibalismo, cultos locos y maníacos religiosos que deben considerarse que se creerán.Las escenas de canibalismo son bastante horribles, pero están tan mal educadas y actuan que demuestran bastante divertidas.El elenco es horrible, excepto Janet, Agra, como la ventaja femenina que es excelente.¡Cuidado con la escena donde se desprende desnuda y cubierta con pintura dorada!Mi calificación - 1 de 10.

Texto limpio:
spoilers esta película llamada comida viva aquí en el reino unido es posiblemente la peor película que haya hecho es brillante solo para eso una chica rica sexy se adapta un hombre de acción robusto para buscar su hermana en la nueva guinea jungle lo que si

In [20]:
import warnings
warnings.filterwarnings("ignore", message="The parameter 'token_pattern' will not be used*")

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    preprocessor = clean_and_lemmatize,  #Aplicamos limpieza + lematización
    tokenizer = tokenizer,
    min_df = 5,
    max_df = 0.9,
    ngram_range = (1, 2)                 # unigramas y bigramas
)

X_train_vect = vectorizer.fit_transform(X_train_text)
X_val_vect = vectorizer.transform(X_val_text)

print("Cantidad de textos en entrenamiento:", X_train_vect.shape[0])
print("Cantidad de palabras distintas en entrenamiento:", X_train_vect.shape[1])
print("\n")

contador_palabras = np.array(X_train_vect.sum(axis = 0)).flatten() #Sumamos frecuencias por palabra
palabras = vectorizer.get_feature_names_out() #Obtenemos las palabras del vocabulario

#Armamos un DataFrame ordenado
freq_df = pd.DataFrame({'palabra': palabras, 'frecuencia': contador_palabras})
freq_df = freq_df.sort_values(by = 'frecuencia', ascending = False)

#Mostramos las 20 más frecuentes
display(freq_df.head(20))

Cantidad de textos en entrenamiento: 40000
Cantidad de palabras distintas en entrenamiento: 158888




,palabra,frecuencia
104323,película,2051.540167
61863,haber,1126.564607
140272,tener,881.016583
151400,ver,880.824419
110756,poder,842.671380
63560,hacer,822.040955
132249,si,605.525391
142483,the,557.692037
66993,historia,545.447317
4624,alguno,526.957019


Previo a establecer y entrenar los modelos, implementamos una función para poder observar las métricas en el conjunto train y test.

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluar(y_true, y_pred, nombre):
    print(f"--- {nombre} ---")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, pos_label='positivo'):.4f}")
    print(f"Recall   : {recall_score(y_true, y_pred, pos_label='positivo'):.4f}")
    print(f"F1 Score : {f1_score(y_true, y_pred, pos_label='positivo'):.4f}")
    print()

## Naive Bayes

## Ensamble híbrido: Stacking

(Multinomial NB, RandomForest, XGBoost) -> Regresion logística (Metamodelo)

In [22]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
!pip install xgboost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

Comenzamos entrenando cada modelo por separado, para luego usarlos con el metamodelo.

Usamos GridSearchCV con 5 folds y almacenamos el mejor estimador para usar con el metamodelo.

El alpha es el Soothing parameter, un tipo de suavizado para mejorar la generalización.

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# MODELO 1: MultinomialNB

nb_param_grid = {
    'alpha': [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1.0, 1.5, 2.0],
    'fit_prior': [True, False]
}

nb_model = MultinomialNB()

nb_search = GridSearchCV(
    nb_model,
    nb_param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

nb_search.fit(X_train_vect, y_train)
best_nb = nb_search.best_estimator_
print("Mejores hiperparámetros MultinomialNB:", nb_search.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Mejores hiperparámetros MultinomialNB: {'alpha': 0.5, 'fit_prior': True}


Seguimos con el segundo modelo, el RandomForest. En este caso, al tener un espacio de búsqueda muy grande, optamos por usar primero RandomizedSearchCV con 5 folds y luego aplicamos GridSearchCV con 5 folds sobre el Top5 combinaciones de hiperparámetros.

Entonces comenzamos con el RandomizedSearchCV con 5 folds.

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pandas as pd

# MODELO 2: RandomForest

    #1ero: RANDOMIZED SEARCH

rf_param_dist = {
    'n_estimators': [200, 300, 400],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_model = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(
    rf_model,
    rf_param_dist,
    n_iter=13,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1,
    random_state=42,
    return_train_score=True
)

random_search.fit(X_train_vect, y_train)

results = pd.DataFrame(random_search.cv_results_)
top5 = results.sort_values(by='rank_test_score').head(5)
top5_params = top5['params'].tolist()

Fitting 5 folds for each of 13 candidates, totalling 65 fits


Observamos el Top5 parámetros para luego usarlos con GridSearchCV:

In [25]:
for param in top5_params:
    print(param)

{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}
{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 30}
{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 30}
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
{'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 20}


Una vez propuesto un espacio más acotado con los mejores hiperparámetros, utilizamos GridSearchCV con 5 folds. Para cada hiperparámetro, le sumamos/restamos valores para probar los valores que están alrededor del mejor hiperparámetro.

In [29]:
# MODELO 2: RandomForest

    #2do: GRID SEARCH
base_params = top5_params[0]

grid_params = {
    'n_estimators': sorted(set([
        max(10, base_params['n_estimators'] - 50),
        base_params['n_estimators'],
        base_params['n_estimators'] + 50
    ])),
    'max_depth': sorted(set([
        base_params['max_depth'] - 5,
        base_params['max_depth'],
        base_params['max_depth'] + 5
    ])),
    'min_samples_split': sorted(set([
        max(2, base_params['min_samples_split'] - 2),
        base_params['min_samples_split'],
        base_params['min_samples_split'] + 2
    ])),
    'min_samples_leaf': sorted(set([
        max(1, base_params['min_samples_leaf'] - 1),
        base_params['min_samples_leaf'],
        base_params['min_samples_leaf'] + 1
    ])),
    'max_features': [base_params['max_features']],
}

print("GridSearch parámetros finales:", grid_params)

grid_search = GridSearchCV(
    rf_model,
    param_grid=grid_params,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_vect, y_train)

best_rf = grid_search.best_estimator_
print("\nMejores hiperparámetros Random Forest (refinados):", grid_search.best_params_)

GridSearch parámetros finales: {'n_estimators': [250, 300, 350], 'max_depth': [25, 30, 35], 'min_samples_split': [2, 4], 'min_samples_leaf': [1, 2], 'max_features': ['sqrt']}
Fitting 3 folds for each of 36 candidates, totalling 108 fits

Mejores hiperparámetros Random Forest (refinados): {'max_depth': 35, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 300}


Finalmente, nos quedamos con la mejor combinación de hiperparámetros para usar en el Stacking!

Para el XGBoost fue necesario encodear para que XGBoost pudiera procesar los datos. Al encodear, no lo hicimos sobre el y_train original, por lo que no fue necesario desencodearlo al final!

Además, aplicamos RandomizedSearchCV con 5 folds ya que el espacio es muy grande. Serían 540 combinaciones. **SI QUEREMOS MEJORAR MÁS EL MODELO PODEMOS VER DE HACER LO DE RANDOM Y DESP GRID!!! O AGARRAR LOS MAS COMUNES DE RANDOM EN UN TOP MAS GRANDE Y USAR GRID CON ESOS :)**

In [ ]:
from sklearn.preprocessing import LabelEncoder

# MODELO 3: XGBoost

#Encodeamos porque sino XGBoost no lo puede procesar
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 6, 10],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1, 0.3, 0.5, 1.0]
}

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

xgb_search = RandomizedSearchCV(
    xgb_model, xgb_param_grid, cv=5, n_iter=8,
    scoring='f1_macro', n_jobs=1, random_state=42
)
xgb_search.fit(X_train_vect, y_train_enc)

best_xgb = xgb_search.best_estimator_
print("Mejores hiperparámetros XGBoost:", xgb_search.best_params_)



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:55:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Finalmente, una vez hallados los mejores hiperparámetros para cada modelo que forma parte del Stacking, procedemos a usarlos.

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# MODELO FINAL: STACKING CON RF, XGB, MNB Y REGRESIÓN LOGÍSTICA COMO METAMODELO

#Tomamos los mejores hiperparámetros de cada modelo para trabajar
estimators = [
    ('nb', best_nb),
    ('rf', best_rf),
    ('xgb', best_xgb)
]

stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(random_state=42),
    passthrough=True,  #esto PODRIA llegar a generar overfitting, podemos probar sacarlo a futuro o no porq tb aporta info al modelo y los profes lo usan en la práctica!
    cv=3, #Se puede aumentar también pero probé bajarlo de 5 a 3 porque quizás generaba overfitting.
    n_jobs=-1
)

#Entrenamos del modelo final
stacking_model.fit(X_train_vect, y_train)

#Evaluamos en train y validación
y_train_pred = stacking_model.predict(X_train_vect)
y_val_pred = stacking_model.predict(X_val_vect)

evaluar(y_train, y_train_pred, "STACKING (NB + RF + XGB) - TRAIN")
evaluar(y_val, y_val_pred, "STACKING (NB + RF + XGB) - VALIDACIÓN")

Predecimos sobre el conjunto de test y preparamos todo para subir a Kaggle.

In [ ]:
# Vectorizamos los datos de test
X_test_vect = vectorizer.transform(test_df['review_es'])

# Predicción del sentimiento
y_test_pred = stacking_model.predict(X_test_vect)

# Construcción del DataFrame de submission
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred
})

# Exportar a CSV
submission.to_csv('stacking_submission.csv', index=False)


In [ ]:
#con esto lo descargamos
from google.colab import files
files.download('stacking_submission.csv')

Redes Neuronales